# Mount ổ Google Drive

In [0]:
try:
    import os
    from google.colab import drive

    drive.mount('/content/drive')
    os.chdir('/content/drive/My Drive/aivivn_vietnamese_tone_prediction/')
    print(f'Current working dir: {os.getcwd()}')
except ImportError:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working dir: /content/drive/My Drive/aivivn_vietnamese_tone_prediction


# Import thư viện

In [0]:
import itertools
import platform
import random
import string
import time
from pathlib import Path
from pprint import pprint
from zipfile import ZipFile

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from model import EncoderRNN, LuongAttnDecoderRNN, SentenceDataSet
from utils import as_minutes, device, get_display, model_dir, n_grams, time_since
from vietnamese_utils import remove_vietnamese_tone, uni_chars_l
from vocab import PAD_token, SOS_token, Vocab

print(f'Python version: {platform.python_version()}')
print(f'Pytorch version: {torch.__version__}')

Python version: 3.6.8
Pytorch version: 1.1.0


# Chuẩn bị bộ từ điển

In [0]:
chars = string.ascii_lowercase
chars += uni_chars_l
chars += ' '

vocab = Vocab()
for c in chars:
    vocab.add_char(c)

print(f'Tổng số ký tự trong từ điển: {vocab.num_words}')

Tổng số ký tự trong từ điển: 98


# Tiền xử lý dữ liệu

## Đọc dữ liệu

In [0]:
# with ZipFile('data/vietnamese_tone_prediction.zip', 'r') as inzip:
#     data = inzip.read('train.txt').decode('utf-8').split('\n')
#     data = data[:500_000]

with open('data/mini_train.txt', 'r', encoding='utf-8') as infile:
    data = infile.read().split('\n')

print(f'Số câu được load: {len(data):_}')
pprint(data[:5])

Số câu được load: 196_341
['Alaska ngày 21 tháng 9 năm 2007',
 'Những kiểu áo sơ mi may theo chất liệu cotton',
 'Sau này mới thấy người ta nói',
 'Giờ Đại tướng đã đi xa',
 'người bị tố cáo lại cũng chủ yếu là các đơn vị']


## Sinh tập training data set



In [0]:
# chuyển tập dữ liệu về lower case
data = [s.lower() for s in data]

# Tách ra theo ngram
train_grams = list(itertools.chain.from_iterable([n_grams(item) for item in data]))

# Sinh tập dữ liệu không dấu
train_rev_accent = [remove_vietnamese_tone(item) for item in train_grams]

pairs = list(zip(train_rev_accent, train_grams))

training_dataset = SentenceDataSet(pairs)

# Print some pairs to validate
print(f'Kích thước tập train: {len(training_dataset):_}')
pprint(training_dataset[:5])

Kích thước tập train: 938_742
[('alaska ngay 21 thang', 'alaska ngày 21 tháng'),
 ('ngay 21 thang 9', 'ngày 21 tháng 9'),
 ('21 thang 9 nam', '21 tháng 9 năm'),
 ('thang 9 nam 2007', 'tháng 9 năm 2007'),
 ('nhung kieu ao so', 'những kiểu áo sơ')]


# Định nghĩa các hàm chuyển text -> vector

In [0]:
# Padding thêm 0 vào list nào có độ dài nhỏ hơn về phía bên phải
def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=vocab.char2index(fillvalue)))


# Tạo ma trận binary có kích thước như ma trận truyền vào l nhưng giá trị của mỗi phần tử đánh dấu 1 hoặc 0 tương ứng với padding hoặc không padding
def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == vocab.char2index(PAD_token):
                m[i].append(0)
            else:
                m[i].append(1)
    return m


# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [voc.sentence2indexes(sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths


# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [voc.sentence2indexes(sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len


# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0]), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

# Training

## Định nghĩa hàm loss

In [0]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [0]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder,
          encoder_optimizer, decoder_optimizer, batch_size, clip):
    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.tensor([[vocab.char2index(SOS_token)] * batch_size], dtype=torch.long)
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.tensor([[topi[i][0] for i in range(batch_size)]], dtype=torch.long)
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals


def trainIters(voc, data_loader,
               embedding, encoder, decoder,
               encoder_optimizer, decoder_optimizer,
               save_dir, batch_size, clip, total_epoch):
    print('Training...')
    epoch_status = get_display('Epoch: 0')
    batch_status = get_display('Batch: 0')
    save_status = get_display('Saved epoch: 0')

    # Training loop
    for epoch in range(1, total_epoch + 1):
        epoch_status.update(f'Training epoch: {epoch}')

        print_loss = 0
        total_batch = 0
        start_batch_time = time.time()
        for it, pairs in enumerate(data_loader, 1):
            if len(pairs[0]) != batch_size:
                continue
            training_batch = batch2TrainData(voc, list(zip(*pairs)))
            # Extract fields from batch
            input_variable, lengths, target_variable, mask, max_target_len = training_batch

            # Run a training iteration with batch
            loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, batch_size, clip)
            s, es, rs = time_since(start_batch_time, it / len(data_loader))
            batch_status.update(f'Batch: {it}/{len(data_loader)}, loss: {loss:.4f},'
                                f' {as_minutes(s)}/{as_minutes(es)}, remain {as_minutes(rs)}, {as_minutes(s / it)}/it')
            print_loss += loss
            total_batch += 1

        # Save checkpoint
        save_dir = Path(save_dir)
        save_dir.mkdir(parents=True, exist_ok=True)
        print_loss_avg = print_loss / total_batch
        save_file = save_dir / f'checkpoint_epoch_{epoch}_loss_{print_loss_avg:.4f}.pt'
        save_content = {
            'epoch': epoch,
            'en': encoder.state_dict(),
            'de': decoder.state_dict(),
            'en_opt': encoder_optimizer.state_dict(),
            'de_opt': decoder_optimizer.state_dict(),
            'loss': loss,
            'voc_dict': voc.__dict__,
            'embedding': embedding.state_dict()
        }
        torch.save(save_content, save_file)
        save_status.update(f'Saved epoch: {epoch}, average loss: {print_loss_avg:.4f}')

In [0]:
# Configure models
attn_model = 'dot'
# attn_model = 'general'
# attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 100
total_epoch = 10

# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0

print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(vocab.num_words, hidden_size)
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, vocab.num_words, decoder_n_layers, dropout)

# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

training_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)

# Run training iterations
print("Starting Training!")
trainIters(vocab, training_loader,
           embedding, encoder, decoder, encoder_optimizer, decoder_optimizer,
           model_dir, batch_size, clip, total_epoch)

Building encoder and decoder ...
Models built and ready to go!
Building optimizers ...
Starting Training!
Training...


'Training epoch: 7'

'Batch: 8105/9388, loss: 0.1038, 32m 59s/38m 12s, remain 5m 13s, 0m 0s/it'

'Saved epoch: 6, average loss: 0.1143'

Buffered data was truncated after reaching the output size limit.